<a href="https://colab.research.google.com/github/HAV-79/Weather-Forecast/blob/main/ConvLSTM_com_MSE%2C_MAE_and_Cross_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras import layers
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import os
from sklearn.model_selection import KFold

from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/dataset Next Frame/conv_lstm_data_1000_128x128_20f.npy'

# Download and load the dataset.
fpath = path
dataset = np.load(fpath)

# Swap the axes representing the number of frames and number of data samples.
dataset = np.swapaxes(dataset, 0, 1)
# We'll pick out 1000 of the 10000 total examples and use those.
dataset = dataset[:1000, ...]
# Add a channel dimension since the images are grayscale.
# dataset = np.expand_dims(dataset, axis=-1)

# Normalize the data to the 0-1 range.
dataset = dataset / 255

# Define a helper function to shift the frames.
def create_shifted_frames(data):
    x = data[:, 0 : data.shape[1] - 1, :, :]
    y = data[:, 1 : data.shape[1], :, :]
    return x, y

# Parameters
epochs = 40
batch_size = 2
n_splits = 2  # Number of folds for K-fold validation

# Create KFold object
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Placeholder for history data
histories = []

for train_index, val_index in kf.split(dataset):
    train_dataset = dataset[train_index]
    val_dataset = dataset[val_index]

    # Apply the processing function to the datasets.
    x_train, y_train = create_shifted_frames(train_dataset)
    x_val, y_val = create_shifted_frames(val_dataset)

    # Construct the input layer with no definite frame size.
    inp = layers.Input(shape=(None, *x_train.shape[2:]))

    # Construct ConvLSTM2D and Conv3D layers.
    x = layers.ConvLSTM2D(
        filters=64,
        kernel_size=(5, 5),
        padding="same",
        return_sequences=True,
        activation="relu",
    )(inp)
    x = layers.BatchNormalization()(x)
    x = layers.ConvLSTM2D(
        filters=64,
        kernel_size=(3, 3),
        padding="same",
        return_sequences=True,
        activation="relu",
    )(x)
    x = layers.BatchNormalization()(x)
    x = layers.ConvLSTM2D(
        filters=64,
        kernel_size=(1, 1),
        padding="same",
        return_sequences=True,
        activation="relu",
    )(x)
    x = layers.Conv3D(
        filters=1, kernel_size=(3, 3, 3), activation="sigmoid", padding="same"
    )(x)

    # Build and compile the model.
    model = keras.models.Model(inp, x)
    model.compile(
        loss='mse',  # Mean Squared Error as loss function
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        metrics=['mae', 'mse']  # Mean Absolute Error and Mean Squared Error as metrics
    )
    model.summary()

    # Define callbacks for early stopping and learning rate reduction.
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

    # Fit the model to the training data and get the history.
    history = model.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(x_val, y_val),
        callbacks=[early_stopping, reduce_lr],
    )

    # Append history for later analysis
    histories.append(history)

# Create a directory to save the model if it doesn't exist.
output_directory = "/content/drive/MyDrive/Modelos"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Save the trained model
model.save(os.path.join(output_directory, 'novo_modelo_1000_40pc_nextframe.h5'))

# Plot combined training & validation loss values from all folds
for i, history in enumerate(histories):
    plt.plot(history.history['loss'], label=f'Train Loss Fold {i+1}')
    plt.plot(history.history['val_loss'], label=f'Validation Loss Fold {i+1}')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()

# Plot combined training & validation MAE values from all folds
for i, history in enumerate(histories):
    plt.plot(history.history['mae'], label=f'Train MAE Fold {i+1}')
    plt.plot(history.history['val_mae'], label=f'Validation MAE Fold {i+1}')
plt.title('Model Mean Absolute Error')
plt.ylabel('MAE')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()

# Plot combined training & validation MSE values from all folds
for i, history in enumerate(histories):
    plt.plot(history.history['mse'], label=f'Train MSE Fold {i+1}')
    plt.plot(history.history['val_mse'], label=f'Validation MSE Fold {i+1}')
plt.title('Model Mean Squared Error')
plt.ylabel('MSE')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()




## Frame Prediction Visualizations - Model trained in 20 epochs.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from google.colab import drive
drive.mount('/content/drive')

# Caminho para o dataset
path = '/content/drive/MyDrive/dataset Next Frame/conv_lstm_data_20_128x128_20f.npy'

# Carregar o dataset
fpath = path
dataset = np.load(fpath)

# Trocar os eixos representando o número de frames e o número de imagens
dataset = np.swapaxes(dataset, 0, 1)

# Vamos escolher 20 imagens
dataset = dataset[:20, ...]

# Normalizar os dados para a faixa 0-1
dataset = dataset / 255.0

# Função auxiliar para deslocar os frames
def create_shifted_frames(data):
    x = data[:, :-1, :, :]
    y = data[:, 1:, :, :]
    return x, y

# Criar dataset de treino e validação (por exemplo, 80% treino e 20% validação)
train_size = int(0.8 * len(dataset))
train_dataset = dataset[:train_size]
val_dataset = dataset[train_size:]

# Carregar o modelo salvo
model_path = '/content/drive/MyDrive/Modelos/modelo_1000_40epc_nextframe_5.h5'
model = tf.keras.models.load_model(model_path)



# Selecionar um exemplo aleatório do dataset de validação
example = val_dataset[np.random.choice(range(len(val_dataset)), size=1)[0]]

# Escolher os primeiros/últimos dez frames do exemplo
frames = example[:10, ...]
original_frames = example[10:, ...]

# Prever um novo conjunto de 10 frames
for _ in range(10):
    # Extrair a predição do modelo e pós-processar
    new_prediction = model.predict(np.expand_dims(frames, axis=0))
    new_prediction = np.squeeze(new_prediction, axis=0)
    predicted_frame = np.expand_dims(new_prediction[-1, ...], axis=0)

    # Estender o conjunto de frames de predição
    frames = np.concatenate((frames, predicted_frame), axis=0)

# Construir uma figura para os frames originais e novos
fig, axes = plt.subplots(2, 10, figsize=(40, 10))

# Plotar os frames originais
for idx, ax in enumerate(axes[0]):
    ax.imshow(np.squeeze(original_frames[idx]), cmap="gray")
    ax.set_title(f"Frame {idx + 11}")
    ax.axis("off")

# Plotar os novos frames
new_frames = frames[10:, ...]
for idx, ax in enumerate(axes[1]):
    ax.imshow(np.squeeze(new_frames[idx]), cmap="gray")
    ax.set_title(f"Frame {idx + 11}")
    ax.axis("off")

# Mostrar a figura
plt.show()

## Videos Forecast saving in Gifs


In [ ]:
import os
import numpy as np
import imageio
import matplotlib.pyplot as plt
import tensorflow as tf
from ipywidgets import HBox, widgets
from IPython.display import display

# Carregar o modelo salvo
model_path = '/content/drive/MyDrive/Modelos/modelo_1000_40epc_nextframe_5.h5'
model = tf.keras.models.load_model(model_path)

# Crie um diretório para salvar os GIFs se ele não existir
output_directory = "Predicted_1000i"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Suponha que val_dataset já esteja definido e carregado corretamente
# Exemplo: val_dataset = np.load('val_dataset.npy')
# (Você deve ajustar esta linha conforme necessário para carregar seu conjunto de dados de validação)

# Selecione alguns exemplos aleatórios do conjunto de dados de validação.
examples = val_dataset[np.random.choice(range(len(val_dataset)), size=5)]

# Iterar sobre os exemplos e prever os frames.
for idx, example in enumerate(examples):
    # Escolha os primeiros/últimos dez frames do exemplo.
    frames = example[:10, ...]
    original_frames = example[10:, ...]
    new_predictions = np.zeros(shape=(10, *frames[0].shape))

    # Prever um novo conjunto de 10 frames.
    for i in range(10):
        # Extraia a previsão do modelo e pós-processe-a.
        frames = example[: 10 + i + 1, ...]
        new_prediction = model.predict(np.expand_dims(frames, axis=0))
        new_prediction = np.squeeze(new_prediction, axis=0)
        predicted_frame = np.expand_dims(new_prediction[-1, ...], axis=0)

        # Estenda o conjunto de frames de previsão.
        new_predictions[i] = predicted_frame

    # Salvar GIFs para cada uma das imagens de verdade/predição.
    for set_name, frame_set in zip(["truth", "prediction"], [original_frames, new_predictions]):
        # Construir um GIF a partir dos frames de vídeo selecionados.
        current_frames = np.squeeze(frame_set)
        if current_frames.ndim == 3:  # Caso haja apenas uma dimensão de cor
            current_frames = current_frames[..., np.newaxis] * np.ones(3)
        current_frames = (current_frames * 255).astype(np.uint8)

        # Salvar o GIF como um arquivo.
        gif_filename = os.path.join(output_directory, f"example_{idx}_set_{set_name}.gif")
        imageio.mimsave(gif_filename, current_frames, "GIF", duration=0.2)

# Exibir os vídeos.
print("Truth\tPrediction")
for i in range(len(examples)):
    # Construir e exibir um `HBox` com a verdade e a previsão.
    truth_gif_path = os.path.join(output_directory, f"example_{i}_set_truth.gif")
    prediction_gif_path = os.path.join(output_directory, f"example_{i}_set_prediction.gif")
    box = HBox(
        [
            widgets.Image(value=open(truth_gif_path, "rb").read(), format='gif'),
            widgets.Image(value=open(prediction_gif_path, "rb").read(), format='gif'),
        ]
    )
    display(box)


# **Make the prediction and save it in png format in a folder**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from google.colab import drive
import os
from PIL import Image

# Montar o Google Drive
drive.mount('/content/drive')

# Caminho para o dataset
path = '/content/drive/MyDrive/dataset Next Frame/conv_lstm_data_20_128x128_20f.npy'

# Carregar o dataset
fpath = path
dataset = np.load(fpath)

# Trocar os eixos representando o número de frames e o número de exemplos de dados
dataset = np.swapaxes(dataset, 0, 1)
# Vamos escolher 200 exemplos do total de exemplos
dataset = dataset[:10, ...]

# Normalizar os dados para a faixa 0-1
dataset = dataset / 255.0

# Função auxiliar para deslocar os frames
def create_shifted_frames(data):
    x = data[:, :-1, :, :]
    y = data[:, 1:, :, :]
    return x, y

# Criar dataset de treino e validação (por exemplo, 80% treino e 20% validação)
train_size = int(0.8 * len(dataset))
train_dataset = dataset[:train_size]
val_dataset = dataset[train_size:]

# Carregar o modelo salvo
model_path = '/content/drive/MyDrive/Modelos/modelo_600i_2opc_2kfolder_nextframe.h5'
model = tf.keras.models.load_model(model_path)

# Selecionar um exemplo aleatório do dataset de validação
example = val_dataset[np.random.choice(range(len(val_dataset)), size=1)[0]]

# Escolher os primeiros/últimos dez frames do exemplo
frames = example[:10, ...]
original_frames = example[10:, ...]

# Definir a pasta onde os novos frames serão salvos
output_dir = '/content/drive/MyDrive/PredictedFrames1000-20epoc'
os.makedirs(output_dir, exist_ok=True)

# Função para salvar um frame como PNG
def save_frame_as_png(frame, filename):
    frame = (frame * 255).astype(np.uint8)  # Convertendo de volta para 0-255
    image = Image.fromarray(frame.squeeze(), 'L')  # 'L' para imagens em escala de cinza
    image.save(filename)

# Prever um novo conjunto de 10 frames
for i in range(10):
    # Extrair a predição do modelo e pós-processar
    new_prediction = model.predict(np.expand_dims(frames, axis=0))
    new_prediction = np.squeeze(new_prediction, axis=0)
    predicted_frame = np.expand_dims(new_prediction[-1, ...], axis=0)

    # Estender o conjunto de frames de predição
    frames = np.concatenate((frames, predicted_frame), axis=0)

    # Salvar o frame predito como PNG
    save_frame_as_png(predicted_frame, os.path.join(output_dir, f'predicted_frame_{i+1}.png'))

# Construir uma figura para os frames originais e novos
fig, axes = plt.subplots(2, 5, figsize=(20, 4))

# Plotar os frames originais
for idx, ax in enumerate(axes[0]):
    ax.imshow(np.squeeze(original_frames[idx]), cmap="gray")
    ax.set_title(f"Frame {idx + 11}")
    ax.axis("off")

# Plotar os novos frames
new_frames = frames[5:, ...]
for idx, ax in enumerate(axes[1]):
    ax.imshow(np.squeeze(new_frames[idx]), cmap="gray")
    ax.set_title(f"Frame {idx + 11}")
    ax.axis("off")

# Mostrar a figura
plt.show()